# CSV to SQL

In [103]:
import pandas as pd
import numpy as np
import sys

In [104]:
def DataframeToCsv(dataframe, fileName, tableName, columns):
    numRow = len(dataframe.index)
    numCol = len(dataframe.columns)
    
    with open(fileName, 'w') as f:
        for row in range(0, numRow):
            print("INSERT INTO " + tableName + " " + columns + " VALUES (", file=f, end="")
            for column in range(0, numCol):
                print("NULL" 
                      if ((dataframe.iat[row, column] == "'nan'")
                          or (dataframe.iat[row, column] == "—"))
                      else dataframe.iat[row, column], file=f, end="")
                if (column != numCol-1):
                    print(", ", file=f, end="")
                else:
                    print(");", file=f)

In [105]:
# PAIS TABLE
# Reads all CSVs
pais = pd.read_csv("raw-data/pais.csv")
capital = pd.read_csv("raw-data/List_of_national_capitals_1.csv")
# So I can get the continent
continents = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv")

# Treats CSV
continents = continents[['Country/Territory', 'UN Region']].copy()
pais.drop("Source and date", axis=1, inplace=True)
pais.drop("Area.1", axis=1, inplace=True)
pais.drop("Population density.1", axis=1, inplace=True)
pais.drop(0, axis=0, inplace=True)
pais.rename({"Country, territory or dependency" : "nome", "Population" : "populacao", "Area" : "area"}, axis=1, inplace=True)
pais = pais.sort_values("nome")
capital.drop("Notes", axis=1, inplace=True)
capital.rename({"Country/Territory" : "nome", "City/Town" : "capital"}, axis=1, inplace=True)
capital.sort_values("nome", inplace=True)
pais = pais.join(capital.set_index('nome'), on='nome')
pais = pais.join(continents.set_index('Country/Territory'), on='nome')
pais['capital'] = pais['capital'].str.replace("'", "''")
pais['populacao'] = pais['populacao'].str.replace(",", "")
pais['area'] = pais['area'].str.replace(",", "")
pais['Population density'] = pais['Population density'].str.replace(",", "")
pais.rename({"UN Region" : "continente"}, axis=1, inplace=True)
pais.update(pais[['nome', 'capital', 'continente']].applymap("'{}'".format))

In [106]:
# PIB TABLE
pib = pd.read_csv("raw-data/List_of_countries_by_GDP_(nominal)_1.csv")
# Treats CSV
pib.drop("UN Region", axis=1, inplace=True)
pib.drop("IMF.1", axis=1, inplace=True)
pib.drop("World Bank.1", axis=1, inplace=True)
pib.drop("United Nations.1", axis=1, inplace=True)
pib.drop(0, axis=0, inplace=True)
pib.drop(1, axis=0, inplace=True)
pib.rename({"Country/Territory" : "pais", "IMF" : "2022", "World Bank" : "2021", "United Nations" : "2020"}, axis=1, inplace=True)
pib['2022'] = pib['2022'].str.replace(",", "")
pib['2021'] = pib['2021'].str.replace(",", "")
pib['2020'] = pib['2020'].str.replace(",", "")
pib.update(pib[['pais']].applymap("'{}'".format))
pib = pib.sort_values("pais")
pib.head()

,pais,2022,2021,2020
120,'Afghanistan',20136,20116,19793
126,'Albania',18256,18260,14910
58,'Algeria',187155,167983,147689
205,'American Samoa',—,709,—
176,'Andorra',3302,3330,2864


In [107]:
DataframeToCsv(pais, "cooked-data/pais.sql", "Pais", "(nome, populacao, area, densidadePop, capital)")
DataframeToCsv(pib, "cooked-data/pib.sql", "PIB", "(nomePais, pib2022, pib2021, pib2020)")